<a href="https://colab.research.google.com/github/Bathilda/AI16_DS_CV/blob/main/CP1_CV_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.9/496.9 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 21.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone

Cloning into 'yolov5'...
remote: Enumerating objects: 15305, done.
remote: Total 15305 (delta 0), reused 0 (delta 0), pack-reused 15305
Receiving objects: 100% (15305/15305), 14.13 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (10501/10501), done.


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 1.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.3
    Uninstalling setuptools-63.4.3:
      Successfully uninstalled setuptools-63.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.3.0 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#json 파일 txt화

import os
import json
from tqdm import tqdm
import shutil
from glob import glob 

img_list=glob('/content/drive/MyDrive/Bootcamp/cp1/New_Sample/원천데이터/원천통합/*.jpg')
json_list = glob('/content/drive/MyDrive/Bootcamp/cp1/New_Sample/라벨링데이터/라벨링통합/*.json')

def make_folders(path="output"):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path

def convert_json_to_yolo_txt(output_path, json_file):
    path = make_folders(output_path)
    for json_path in json_list:
        with open(json_path) as f:
            json_data = json.load(f)
            # label_file = os.path.join(output_path, "_darknet.labels")
            json_basename = os.path.basename(json_path)  # JSON 파일 이름
            txt_basename = os.path.splitext(json_basename)[0] + '.txt'  # TXT 파일 이름
                    
            # with open(label_file, "w") as f:
            with open(os.path.join(output_path, txt_basename), "w") as f:  
              # for i in json_data['items']:
              for file_dict in tqdm(json_data["FILE"]):
                items_list = file_dict["ITEMS"]
                resol = file_dict['RESOLUTION'].split('*')
                WI = float(resol[0])
                HI = float(resol[1])
                for item_dict in items_list:
                  yolov5_format_list = []
                  package = item_dict['PACKAGE']
                  #매핑
                  classmap = {'정상차량':0,'불법차량':1}
                  classid = classmap[package]
                  #바운딩 박스 좌표를 이용, yolo형식의 라벨 형성(정규화)
                  box = item_dict['BOX'].split(',')
                  x1 = float(box[0])
                  y1 = float(box[1])
                  w = float(box[2])
                  h = float(box[3])
                  xcent = (x1+w/2)/WI
                  ycent = (y1+h/2)/HI
                  width = w/WI
                  height = h/HI
                  yolov5_format = [classid, xcent, ycent, width, height]
                  f.write(' '.join(map(str, yolov5_format)) + '\n')

convert_json_to_yolo_txt('/content/dataset/labels', json_list)

100%|██████████| 1/1 [00:00<00:00, 9642.08it/s]


In [ ]:
from glob import glob 
img_list=glob('/content/dataset/images/*.jpg') 
print(len(img_list))

# train(80%)과 val(20%)로 구분
from sklearn.model_selection import train_test_split
train_img_list, val_img_list=train_test_split(img_list, test_size=0.2, random_state=2000) 
print(len(train_img_list), len(val_img_list))

%cat /content/dataset/data.yml


340
272 68
train: ../train/images
val: ../valid/images
nc: 2
names:
  - normal
  - over-loaded

In [ ]:

with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')
with open('/content/dataset/val.txt', 'w') as f: 
  f.write('\n'.join(val_img_list)+'\n')

import yaml
with open("/content/dataset/data.yml","r") as f:
  data = yaml.safe_load(f)


data['train'] = '/content/dataset/train.txt' 
data['val'] = '/content/dataset/val.txt'

with open("/content/dataset/data.yml","w") as f:
  yaml.dump(data,f)


#data_config.yaml파일의 내용을 수정
%cat /content/dataset/data.yml
#수정된 data_config.yaml확인

names:
- normal
- over-loaded
nc: 2
train: /content/dataset/train.txt
val: /content/dataset/val.txt


In [ ]:
%cd /content/
!python yolov5/train.py --img 416 --batch 16 --epochs 200 --cfg yolov5/models/yolov5s.yaml --data /content/dataset/data.yml --weights yolov5s.pt --cache


/content
train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/content/dataset/data.yml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, 

In [ ]:
%cd /content/
!python yolov5/detect.py --source sample.mp4 --weights yolov5/runs/train/exp/weights/best.pt

/content
detect: weights=['yolov5/runs/train/exp/weights/best.pt'], source=sample.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/915) /content/sample.mp4: 448x640 1 normal, 12.4ms
video 1/1 (2/915) /content/sample.mp4: 448x640 1 normal, 10.2ms
video 1/1 (3/915) /content/sample.mp4: 448x640 1 normal, 10.2ms
video 1/1 (4/915) /content/sample.mp4: 448x640 1 normal, 9.9ms
video 1/1 (5/915) /content/sample.mp4: 448x640 2 normals, 9.9ms
vide

In [ ]:
%cd /content/
!python yolov5/export.py --weights yolov5/runs/train/exp/weights/best.pt --include onnx

/content
export: data=yolov5/data/coco128.yaml, weights=['yolov5/runs/train/exp/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from yolov5/runs/train/exp/weights/best.pt with output shape (1, 25200, 7) (13.6 MB)
requirements: YOLOv5 requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which 

# roboflow 활용한 데이터 증강

In [ ]:
from glob import glob 
train_img_list=glob('/content/dataset_ag/train/images/*.jpg') 
val_img_list=glob('/content/dataset_ag/valid/images/*.jpg') 

with open('/content/dataset_ag/train/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')
with open('/content/dataset_ag/valid/val.txt', 'w') as f:  
  f.write('\n'.join(val_img_list)+'\n')

import yaml
with open("/content/dataset_ag/data.yaml","r") as f:
  data = yaml.safe_load(f)


data['train'] = '/content/dataset_ag/train/train.txt' 
data['val'] = '/content/dataset_ag/valid/val.txt'

with open("/content/dataset_ag/data.yaml","w") as f:
  yaml.dump(data,f)

In [ ]:
#data_config.yaml파일의 내용을 수정
%cat /content/dataset_ag/data.yaml
#수정된 data_config.yaml확인

names:
- normal
- over-loaded
nc: 2
roboflow:
  license: CC BY 4.0
  project: overloaded-car
  url: https://universe.roboflow.com/project/overloaded-car/dataset/1
  version: 1
  workspace: project
test: ../test/images
train: /content/dataset_ag/train/train.txt
val: /content/dataset_ag/valid/val.txt


In [ ]:
%cd /content/
!python yolov5/train.py --img 416 --batch 16 --epochs 200 --cfg yolov5/models/yolov5s.yaml --data /content/dataset_ag/data.yaml --weights yolov5s.pt --cache


/content
train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/content/dataset_ag/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0

In [ ]:
%cd /content/
!python yolov5/detect.py --source sample.mp4 --weights yolov5/runs/train/exp2/weights/best.pt

/content
detect: weights=['yolov5/runs/train/exp2/weights/best.pt'], source=sample.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/915) /content/sample.mp4: 448x640 1 normal, 12.9ms
video 1/1 (2/915) /content/sample.mp4: 448x640 1 normal, 9.9ms
video 1/1 (3/915) /content/sample.mp4: 448x640 1 normal, 12.9ms
video 1/1 (4/915) /content/sample.mp4: 448x640 1 normal, 9.8ms
video 1/1 (5/915) /content/sample.mp4: 448x640 2 normals, 9.8ms
vide

In [ ]:
%cd /content/
!python yolov5/export.py --weights yolov5/runs/train/exp2/weights/best.pt --include onnx

/content
export: data=yolov5/data/coco128.yaml, weights=['yolov5/runs/train/exp2/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from yolov5/runs/train/exp2/weights/best.pt with output shape (1, 25200, 7) (13.6 MB)

ONNX: starting export with onnx 1.13.1...
ONNX: export success ✅ 1.2s, saved as yolov5/runs/train/exp2/weights/best.onnx (27.2 MB)

Export complete (2.7s)
Results saved to /content/yolov5/runs/train/exp2/weights
Detect:          python detect.py --weights yolov5/runs/train/exp2/weights/best.onnx 
Validate:        python val.py --weights yo

In [ ]:
%cd /content/
!python yolov5/export.py --weights yolov5/runs/train/exp2/weights/best.pt --include onnx

/content
export: data=yolov5/data/coco128.yaml, weights=['yolov5/runs/train/exp2/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from yolov5/runs/train/exp2/weights/best.pt with output shape (1, 25200, 7) (13.6 MB)

ONNX: starting export with onnx 1.13.1...
ONNX: export success ✅ 1.7s, saved as yolov5/runs/train/exp2/weights/best.onnx (27.2 MB)

Export complete (3.9s)
Results saved to /content/yolov5/runs/train/exp2/weights
Detect:          python detect.py --weights yolov5/runs/train/exp2/weights/best.onnx 
Validate:        python val.py --weights yo